In [2]:
import sys

from snappy import GPF
from snappy import HashMap
from snappy import ProductIO
from snappy import jpy
from osgeo import gdal

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [4]:
file = '/eodata/Envisat/Meris/FRS/2012/04/08/MER_FRS_1PPEPA20120408_105857_000005063113_00267_52867_0978.N1'

print("Reading...")
product = ProductIO.readProduct(file)
width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

print("Product: %s, %d x %d pixels, %s" % (name, width, height, description))
print("Bands:   %s" % (list(band_names)))

Reading...
Product: MER_FRS_1PPEPA20120408_105857_000005063113_00267_52867_0978.N1, 4481 x 11521 pixels, MERIS Full Resolution Geolocated and Calibrated TOA Radiance
Bands:   ['radiance_1', 'radiance_2', 'radiance_3', 'radiance_4', 'radiance_5', 'radiance_6', 'radiance_7', 'radiance_8', 'radiance_9', 'radiance_10', 'radiance_11', 'radiance_12', 'radiance_13', 'radiance_14', 'radiance_15', 'l1_flags', 'detector_index']


In [5]:
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand1 = BandDescriptor()
targetBand1.name = 'band_1'
targetBand1.type = 'float32'
targetBand1.expression = '(radiance_10 - radiance_7) / (radiance_10 + radiance_7)'

targetBand2 = BandDescriptor()
targetBand2.name = 'band_2'
targetBand2.type = 'float32'
targetBand2.expression = '(radiance_9 - radiance_6) / (radiance_9 + radiance_6)'

targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 2)
targetBands[0] = targetBand1
targetBands[1] = targetBand2

In [6]:
parameters = HashMap()
parameters.put('targetBands', targetBands)

result = GPF.createProduct('BandMaths', parameters, product)

print("Writing...")

ProductIO.writeProduct(result, 'snappy_bmaths_output', 'GeoTIFF')
# ProductIO.writeProduct(result, 'snappy_bmaths_output.dim', 'BEAM-DIMAP') 
result.closeIO()

print("Done.")

Writing...
Done.


In [7]:
options_list = ['-ot Byte', '-of JPEG', '-b 1', '-scale'] 
options_string = " ".join(options_list)

gdal.Translate('snappy_bmaths_output.jpg', 'snappy_bmaths_output.tif', options=options_string)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f42df75fea0> >

In [8]:
layer=ImageLayer()
box=layer.getbb(file)
print(box)
layer.addImageLayer('snappy_bmaths_output.jpg', box, 'Snappy-bmaths')
layer.showLayer()

org.esa.snap.core.datamodel.Product[name=MER_FRS_1PPEPA20120408_105857_000005063113_00267_52867_0978.N1]
63.239086
[[63.239086,-15.790602],[30.949018,12.908287]]
